In [16]:
from gurobipy import Model, quicksum, GRB

In [17]:
city_names = ["Aachen", "Dortmund", "Köln"]
label_widths = [len(city) for city in city_names]
label_heights = [3 for _ in city_names]

x_pos = [21,25,28]
y_pos = [5, 5, 5]

assert(len(city_names) == len(x_pos) == len(y_pos) == len(label_heights) == len(label_widths))


I = len(city_names)
J = 4 #number of orientations

######### ORIENTATION DEFINITION ############
# 0 = NE, 1 = SE, 2 = SW, 3 = NW

W, H = 50,15

print(list(zip(city_names, label_widths, label_heights, x_pos, y_pos)))

# assert that every city is within map:
for i in range(I):
    assert(0 <= x_pos[i] < W)
    assert(0 <= y_pos[i] < H)


[('Aachen', 6, 3, 21, 5), ('Dortmund', 8, 3, 25, 5), ('Köln', 4, 3, 28, 5)]


In [18]:
def get_x_intervall(i, j):
    if j in (0,1): #NE or SE
        return set_from_intervall((x_pos[i] +1, x_pos[i] + label_widths[i]))
    elif j in (2, 3):
        return set_from_intervall((x_pos[i] - label_widths[i], x_pos[i]-1))

def get_y_intervall(i,j):
    if j in (0, 3): # its north
        return set_from_intervall((y_pos[i] - label_heights[i], y_pos[i] -1, ))
    elif j in (1,2): # iths south
        return set_from_intervall((y_pos[i] + 1,  y_pos[i] + label_heights[i], ))

def set_from_intervall(intervall):
    return set(range(intervall[0], intervall[1] +1 ))


In [19]:
# notation for label orientation (clock-wise definition):
# 0 = NE / 1=SE, 2=SW, 3=NW

def labels_are_intersecting(n, m):
    a_x = get_x_intervall(*n)
    a_y = get_y_intervall(*n)

    b_x = get_x_intervall(*m)
    b_y = get_y_intervall(*m)
    return len(a_x & b_x) != 0 or len(a_y & b_y) != 0    

# model intersecting labels as graph: nodes are every label: (i,j)
# adjacent nodes means: Label are intersecting

labels = [(i,j) for i,c in enumerate(city_names) for j in range(J)]

import itertools
intersections = [
    (a,b) for a,b in itertools.combinations(labels, 2) if labels_are_intersecting(a,b) == True
]

print(len(intersections))
print(intersections)

50
[((0, 0), (0, 1)), ((0, 0), (0, 3)), ((0, 0), (1, 0)), ((0, 0), (1, 1)), ((0, 0), (1, 2)), ((0, 0), (1, 3)), ((0, 0), (2, 0)), ((0, 0), (2, 2)), ((0, 0), (2, 3)), ((0, 1), (0, 2)), ((0, 1), (1, 0)), ((0, 1), (1, 1)), ((0, 1), (1, 2)), ((0, 1), (1, 3)), ((0, 1), (2, 1)), ((0, 1), (2, 2)), ((0, 1), (2, 3)), ((0, 2), (0, 3)), ((0, 2), (1, 1)), ((0, 2), (1, 2)), ((0, 2), (1, 3)), ((0, 2), (2, 1)), ((0, 2), (2, 2)), ((0, 3), (1, 0)), ((0, 3), (1, 2)), ((0, 3), (1, 3)), ((0, 3), (2, 0)), ((0, 3), (2, 3)), ((1, 0), (1, 1)), ((1, 0), (1, 3)), ((1, 0), (2, 0)), ((1, 0), (2, 1)), ((1, 0), (2, 2)), ((1, 0), (2, 3)), ((1, 1), (1, 2)), ((1, 1), (2, 0)), ((1, 1), (2, 1)), ((1, 1), (2, 2)), ((1, 1), (2, 3)), ((1, 2), (1, 3)), ((1, 2), (2, 1)), ((1, 2), (2, 2)), ((1, 2), (2, 3)), ((1, 3), (2, 0)), ((1, 3), (2, 2)), ((1, 3), (2, 3)), ((2, 0), (2, 1)), ((2, 0), (2, 3)), ((2, 1), (2, 2)), ((2, 2), (2, 3))]


In [20]:
map_ = [
    [" " for _ in range(W)] for _ in range(H)
]

def print_map():
    print("-"*(W +2))
    for row in map_:
        row_string = "|" + "".join(row) + "|"
        print(row_string)
    print("-"*(W +2))

# mark every city with 'x'
for i in range(I):
    x,y = x_pos[i], y_pos[i]
    map_[y][x] = "x"

print("Thats the map prior the label optimization.")
print_map()




Thats the map prior the label optimization.
----------------------------------------------------
|                                                  |
|                                                  |
|                                                  |
|                                                  |
|                                                  |
|                     x   x  x                     |
|                                                  |
|                                                  |
|                                                  |
|                                                  |
|                                                  |
|                                                  |
|                                                  |
|                                                  |
|                                                  |
----------------------------------------------------


In [21]:
m = Model("Map Labeling Problem")

x = m.addVars(I, J, vtype=GRB.BINARY, name="x")

In [22]:
# At most, there is one label printed for each city.
m.addConstrs(
    quicksum(x[i, j] for j in range(J)) <= 1 for i in range(I)
)

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>}

In [23]:
# two labels must not overlap!
# edges containts adjacent graph edges --> adjacent Labels (nodes of this graph) intersect

for (i1, j1), (i2, j2) in intersections:
    ...
    m.addConstr(
        x[i1, j1] + x[i2, j2] <= 1
    )





In [24]:
# maximize the number of labeled cities:

m.setObjective(
    quicksum(x[i,j] for i in range(I) for j in range(J)),
    GRB.MAXIMIZE
)


In [25]:
m.update()
m.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 53 rows, 12 columns and 112 nonzeros
Model fingerprint: 0x4a0abd69
Variable types: 0 continuous, 12 integer (12 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 2.0000000
Presolve removed 49 rows and 8 columns
Presolve time: 0.00s
Presolved: 4 rows, 4 columns, 8 nonzeros
Found heuristic solution: objective 2.0000000
Variable types: 0 continuous, 4 integer (4 binary)

Root relaxation: cutoff, 0 iterations, 0.00 seconds (0.00 work units)

Explored 1 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 8 (of 8 available processors)

Solution count 2: 2 2 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.000000000000e+00, best bound 2.000

In [26]:
for i, city in enumerate(city_names):

    for j in range(J):
        print("Var x[i,j] = ", x[i,j])

Var x[i,j] =  <gurobi.Var x[0,0] (value 0.0)>
Var x[i,j] =  <gurobi.Var x[0,1] (value 0.0)>
Var x[i,j] =  <gurobi.Var x[0,2] (value 0.0)>
Var x[i,j] =  <gurobi.Var x[0,3] (value 1.0)>
Var x[i,j] =  <gurobi.Var x[1,0] (value 0.0)>
Var x[i,j] =  <gurobi.Var x[1,1] (value 0.0)>
Var x[i,j] =  <gurobi.Var x[1,2] (value 0.0)>
Var x[i,j] =  <gurobi.Var x[1,3] (value 0.0)>
Var x[i,j] =  <gurobi.Var x[2,0] (value 0.0)>
Var x[i,j] =  <gurobi.Var x[2,1] (value 1.0)>
Var x[i,j] =  <gurobi.Var x[2,2] (value 0.0)>
Var x[i,j] =  <gurobi.Var x[2,3] (value 0.0)>


In [27]:
def add_label_to_map(i,j):
    ...
    label = city_names[i]
    label_width = label_widths[i]
    label_height = label_heights[i]
    # 0=NE, 1=SE, 2=SW 3=NW
    x, y = x_pos[i], y_pos[i]

    if j in (0,1):
        x_start = x + 1
    else:
        x_start = x - label_width

    if j in (0, 3): #north
        y_start = y - label_height
    else:
        y_start = y + 1

    try:
        for i in range(x_start, x_start + label_width):
            map_[y_start][i] = "_"
    
    
        for i in range(x_start, x_start + label_width):
            map_[y_start +2][i] = "-"
        
        for index, char in enumerate(label):
            map_[y_start + 1][x_start + index] = char

    except:
        ...    
    


In [28]:
# add_label_to_map(2,2)

for i in range(I):
    for j in range(J):
        if x[i,j].X == 1:
            print("Found active label: x_", city_names[i], "_",j)
            add_label_to_map(i,j)

            

Found active label: x_ Aachen _ 3
Found active label: x_ Köln _ 1


In [29]:
print_map()

----------------------------------------------------
|                                                  |
|                                                  |
|               ______                             |
|               Aachen                             |
|               ------                             |
|                     x   x  x                     |
|                             ____                 |
|                             Köln                 |
|                             ----                 |
|                                                  |
|                                                  |
|                                                  |
|                                                  |
|                                                  |
|                                                  |
----------------------------------------------------
